In [ ]:
import arya
from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
from astropy.nddata import CCDData

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u

In [ ]:

import sys
sys.path.append("..")
sys.path.append("../../imaging/")
from phot_utils import show_image
from convenience_functions import cmap

In [ ]:
coadd = CCDData.read(f"../yasone2/stacked_r/coadd.fits", unit="adu")
coadd_weights = CCDData.read(f"../yasone2/stacked_r/coadd.weight.fits", unit="adu")

frame = CCDData.read(f"../yasone2/img_r_01/nobkg.fits", unit="adu")
frame_weights = CCDData.read(f"../yasone2/img_r_01/nobkg.weight.fits", unit="adu")
frame_bkg_err = CCDData.read(f"../yasone2/img_r_01/bkg_err.fits", unit="adu")


In [ ]:
bg_thresh = 40

In [ ]:
sigma_frame = 1 / np.sqrt(frame_weights.data)
sigma_coadd = 1 / np.sqrt(coadd_weights.data)

In [ ]:
np.median(frame_bkg_err)

In [ ]:
np.sqrt(14)

In [ ]:
plt.figure()
plt.hist(sigma_frame[sigma_frame < np.inf], bins=np.linspace(0, 600, 100))
plt.hist(sigma_coadd[sigma_coadd < np.inf], bins=np.linspace(0, 200, 100), histtype="step");


In [ ]:
plt.figure()
plt.hist(sigma_frame[sigma_frame < np.inf], bins=np.linspace(0, 600, 100))
plt.hist(sigma_coadd[sigma_coadd < np.inf], bins=np.linspace(0, 200, 100), histtype="step");
plt.yscale("log")

In [ ]:
np.median(sigma_frame) / np.median(sigma_coadd)

In [ ]:
np.sqrt(14)

In [ ]:
show_image(coadd_weights)

In [ ]:
coadd_mask = coadd_weights.data > 0

coadd_clean = CCDData(coadd.data * coadd_mask, unit="adu")


In [ ]:
show_image(frame, log=True)

In [ ]:
show_image(coadd, log=True)

In [ ]:
show_image(coadd_clean.data / bg_thresh , log=False, clim=(0, 10), clabel="sigma above bg")

In [ ]:
ra0, dec0 = 262.34921, 6.42302

In [ ]:
coord0 = SkyCoord(ra0, dec0, unit=u.degree)
r_0 = 25 / 0.254 

In [ ]:
yasone2 = Table.read("../yasone2/stacked_r/detection.cat", format="fits", hdu=2)
len(yasone2)
coadd = CCDData.read(f"../yasone2/stacked_r/coadd.fits", unit="adu")


In [ ]:
plt.figure()

plt.scatter(yasone2["ALPHA_J2000"], yasone2["DELTA_J2000"], s=1)
plt.scatter(ra0, dec0)

In [ ]:
%matplotlib ipympl

In [ ]:
plt.figure()
plt.hist(coadd.data.flatten(), bins=np.linspace(0, 62_000, 100))
plt.yscale("log")

In [ ]:

filt = yasone2["FLAGS_WEIGHT"] == 0
filt &= (yasone2["FLAGS"] & (4 + 8 + 16 + 32 + 64 + 128)) == 0
filt &= yasone2["IMAFLAGS_ISO"] == 0

yasone2_unsat = yasone2[filt]
yasone2_biased = yasone2[~filt]
yasone2_sat = yasone2[(yasone2["IMAFLAGS_ISO"] & 1 > 0) | (yasone2["FLAGS"] & 4 > 0)]
yasone2_nearby = yasone2[(yasone2["FLAGS"] & 1 + 2 > 0)]

In [ ]:
plt.figure()
plt.scatter(yasone2_unsat["MAG_APER"][:, 5], yasone2_unsat["MAGERR_APER"][:, 5])
plt.xlim(-20, 0)
plt.ylim(0, 1)

In [ ]:
def plot_yasone(yasone, **kwargs):
    plt.scatter(yasone["XWIN_IMAGE"] - 1, yasone["YWIN_IMAGE"] - 1, s=0.5, lw=0, **kwargs)

In [ ]:
from matplotlib.patches import Ellipse

In [ ]:
def plot_sources(objects, scale=1, color="red", label="", **kwargs):
    ax = plt.gca()
    
    # plot an ellipse for each object
    for i in range(len(objects)):
        e = Ellipse(xy=(objects['X_IMAGE'][i]-1, objects['Y_IMAGE'][i]-1),
                    width=scale*objects['A_IMAGE'][i],
                    height=scale*objects['B_IMAGE'][i],
                    angle=objects['THETA_IMAGE'][i] * 180. / np.pi, linewidth=0.5, **kwargs)

        if i == 0:
            e.set_label(label)
        e.set_facecolor('none')
        e.set_edgecolor(color)
        ax.add_artist(e)
        # plt.scatter(objects['XWIN_IMAGE'][i]-1, objects['YWIN_IMAGE'][i]-1, s=0.5, linewidth=0, color=color)


In [ ]:
arya.COLORS[8]

In [ ]:
fig, ax = plt.subplots(subplot_kw=dict(projection=coadd.wcs), dpi=400, figsize=(5, 5))

plt.imshow(coadd_clean, clim=(40, 1e4), norm="log", cmap=cmap)
plot_sources(yasone2, color="red", label="good detections")
plot_sources(yasone2_biased, color=arya.COLORS[8], label="bad")
plot_sources(yasone2_nearby, color=arya.COLORS[1], label="nearby/blended")
plot_sources(yasone2_sat, color=arya.COLORS[4], label="saturated")

ax.add_patch(
    plt.Circle(coadd.wcs.world_to_pixel(coord0), r_0, color="black", fill=False, edgecolor="black")
)
arya.Legend(-1)
plt.tight_layout()

In [ ]:
np.median(yasone2["FWHM_WORLD"][yasone2["FLAGS"] == 0].to("arcsec"))

In [ ]:
plt.hist(yasone2["FWHM_WORLD"][yasone2["FLAGS"] == 0].to("arcsec"))